In [1]:
library(Seurat)
library(tidyverse)
library(ggplot2)
library(patchwork)
library(forcats)
library(Polychrome)
library(grid)

Loading required package: SeuratObject

Loading required package: sp


Attaching package: ‘SeuratObject’


The following object is masked from ‘package:base’:

    intersect


── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.2     ✔ purrr   0.3.4
✔ tibble  3.2.1     ✔ dplyr   1.1.2
✔ tidyr   1.2.1     ✔ stringr 1.4.1
✔ readr   2.1.2     ✔ forcats 0.5.2
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
xenium.obj = qs::qread('_targets/objects/obj_merged')
merged_meta = qs::qread('_targets/objects/obj_rctd_merged_01') %>% `[[`
selected_cells = merged_meta %>% rownames
xenium.obj = xenium.obj %>% subset(cells = selected_cells) %>% AddMetaData(merged_meta)
xenium.obj


Warning message:
“Not validating FOV objects”
Warning message:
“Not validating Centroids objects”
Warning message:
“Not validating Centroids objects”
Warning message:
“Not validating FOV objects”
Warning message:
“Not validating Centroids objects”
Warning message:
“Not validating FOV objects”
Warning message:
“Not validating FOV objects”
Warning message:
“Not validating FOV objects”
Warning message:
“Not validating Seurat objects”
Warning message:
“Not validating FOV objects”
Warning message:
“Not validating Centroids objects”
Warning message:
“Not validating Centroids objects”
Warning message:
“Not validating FOV objects”
Warning message:
“Not validating Centroids objects”
Warning message:
“Not validating FOV objects”
Warning message:
“Not validating FOV objects”
Warning message:
“Not validating FOV objects”
Warning message:
“Not validating Seurat objects”
Warning message:
“Not validating FOV objects”
Warning message:
“Not validating Centroids objects”
Warning message:
“Not validating

An object of class Seurat 
400 features across 28082 samples within 4 assays 
Active assay: Xenium (100 features, 0 variable features)
 1 layer present: counts
 3 other assays present: BlankCodeword, ControlCodeword, ControlProbe
 8 spatial fields of view present: fov fov.2 fov.3 fov.4 fov.5 fov.6 fov.7 fov.8

In [8]:
xenium.obj@meta.data = xenium.obj@meta.data %>%
mutate(predicted.polar_label = polar_label)

In [9]:
rownames(xenium.obj)

[1] "Sorcs1"   "Lingo2"   "Cntn4"    "Tenm2"    "Sox5"     "Ptprd"   
  [7] "Robo1"    "Plcl1"    "Oxr1"     "Plxdc2"   "Plp1"     "Kirrel3" 
 [13] "Mgat4c"   "Tenm3"    "Slit2"    "Nrg3"     "Hdac9"    "Col25a1" 
 [19] "Ncam2"    "Rbfox1"   "Fgf13"    "Pde10a"   "Deptor"   "Alcam"   
 [25] "Gria1"    "Grid2"    "Fam155a"  "Grm7"     "Agrp"     "Naaladl2"
 [31] "Ntm"      "Gpc5"     "Hs3st4"   "Lef1"     "Erbb4"    "Kctd16"  
 [37] "Zfhx3"    "Gtdc1"    "Pdgfra"   "Nrg1"     "Macrod2"  "Gpc6"    
 [43] "Pcdh11x"  "Kcnip4"   "Brinp3"   "Cadm2"    "Pde4b"    "Fhit"    
 [49] "Rgs6"     "Prkg1"    "Dlg2"     "Ptprk"    "Luzp2"    "Ptprt"   
 [55] "Nxph1"    "Kcnq3"    "Lrmda"    "Dcc"      "Cntn5"    "Adarb2"  
 [61] "Gabrg3"   "Rmst"     "Lrp1b"    "Trpm3"    "P3h2"     "Prr16"   
 [67] "Prkca"    "Nrxn3"    "Aqp4"     "Ctnna2"   "Inpp4b"   "Sgcz"    
 [73] "Fat3"     "Plcb1"    "Cacna2d3" "Bmp4"     "Tox"      "Car10"   
 [79] "Nkain2"   "Galntl6"  "Slc7a11"  "Pcdh9"    "Slc1a2"   "Pdzrn3"  
 [85] "Rax"      "Pomc"     "Zfp804b"  "Wwox"     "Pcdh7"    "Grm8"    
 [91] "Rtl4"     "Il1rapl2" "Sgcd"     "Egfem1"   "Arhgap6"  "Immp2l"  
 [97] "Cfap299"  "Htr3b"    "Csmd1"    "Lmx1a"

In [10]:
create_grouping_col <- function(seurat_obj, prefix) {
  new_metadata <- case_when(str_detect(seurat_obj@meta.data$predicted.polar_label, prefix) ~ seurat_obj@meta.data$predicted.polar_label,
                            TRUE ~ NA_character_) %>% as.factor
  new_metadata = new_metadata %>% fct_relevel(sort(na.omit(levels(new_metadata))), after = Inf)
  return(new_metadata)
}



In [11]:
plot_celltype_idp = function(seurat_obj, cell_str, fov='fov'){
    new_grouping = seurat_obj %>%
        create_grouping_col(cell_str)
    seurat_obj = AddMetaData(seurat_obj, new_grouping, col.name = 'grouping_col')

    # Get the levels of the lab_Pomc column, including the NA_character_ level
    grouping_levels <- levels(seurat_obj@meta.data$grouping_col)
    # Generate colors using the polychrome palette
    how_many_colors = length(grouping_levels)
    colors <- Polychrome::sky.colors(how_many_colors) %>% as.character %>% `[`(1:how_many_colors) # Excluding the NA_character_ level
    # Add 'gray10' for the NA_character_ level
    colors <- c(colors, "gray10")
    # Create a named vector of colors, with levels as names
    named_colors <- setNames(colors, grouping_levels)
    # Map the lab_Pomc column values to the corresponding colors
    # color_vector <- mapvalues(xenium.obj@meta.data$lab_Pomc, from = lab_Pomc_levels, to = named_colors)
    
    ggp = ImageDimPlot(seurat_obj,
                       group.by='grouping_col',
                       boundaries = 'segmentation',
                       fov = fov,
                       border.size =0.1, 
                       border.color =NA,
                       na.value='gray20', 
                       cols=named_colors, 
                       axes = FALSE,
                       size=0)
    ggp
}

In [17]:
make_idp_pdf = function(seurat_obj, cell_type, prefix="^", suffix="\\.(neg|none|pos)", pdf_folder=""){
    if (!file.exists(pdf_folder)) {
      dir.create(pdf_folder, showWarnings = F)
    }
    cell_type_regex = cell_type %>% paste0(prefix, ., suffix)
    dir.create(pdf_folder, recursive = TRUE, showWarnings = FALSE)
    a1 = plot_celltype_idp(seurat_obj, cell_type_regex, fov='fov')  + theme(legend.position = "none")
    a2 = plot_celltype_idp(seurat_obj, cell_type_regex, fov='fov.2') + theme(legend.position = "none")
    b1 = plot_celltype_idp(seurat_obj, cell_type_regex, fov='fov.3') + theme(legend.position = "none")
    b2 = plot_celltype_idp(seurat_obj, cell_type_regex, fov='fov.4') + theme(legend.position = "none")
    c1 = plot_celltype_idp(seurat_obj, cell_type_regex, fov='fov.5') + theme(legend.position = "none")
    c2 = plot_celltype_idp(seurat_obj, cell_type_regex, fov='fov.6') + theme(legend.position = "none")
    d1 = plot_celltype_idp(seurat_obj, cell_type_regex, fov='fov.7') + theme(legend.position = "none")
    d2 = plot_celltype_idp(seurat_obj, cell_type_regex, fov='fov.8') + theme(legend.position = "none")
    title_theme <- theme(plot.title = element_text(color = "white", size = 8, hjust = 0.5))
    # Set black background theme for each plot
    legend_theme <- theme(legend.position = "bottom",
                          legend.direction = "horizontal",
                          legend.box = "horizontal",
                          legend.text = element_text(size = 4),
                          legend.key.size = unit(2, "mm"))
    # black_background_theme <- theme(plot.background = element_rect(fill = "black"),
    #                                 panel.background = element_rect(fill = "black"))
    black_background_theme <- theme(plot.background = element_rect(fill = "black", color = 'black'),
                          panel.background = element_rect(fill = "black", color = 'black'),
                          plot.margin = margin(0, 0, 0, 0),
                          panel.grid.major = element_blank(),
                          panel.grid.minor = element_blank(),
                          panel.border = element_blank())

    a1_with_title <- a1 + theme(legend.position = "right") + ggtitle("a1_Veh_PF") + title_theme + legend_theme + black_background_theme
    a2_no_legend_with_title <- a2 + guides(scale="none") + ggtitle("a2_Veh_PF") + title_theme + black_background_theme
    b1_no_legend_with_title <- b1 + guides(scale="none") + ggtitle("b1_Veh_PF") + title_theme + black_background_theme
    b2_no_legend_with_title <- b2 + guides(scale="none") + ggtitle("b2_Veh_PF") + title_theme + black_background_theme
    c1_no_legend_with_title <- c1 + guides(scale="none") + ggtitle("c1_FGF1") + title_theme + black_background_theme
    c2_no_legend_with_title <- c2 + guides(scale="none") + ggtitle("c2_FGF1") + title_theme + black_background_theme
    d1_no_legend_with_title <- d1 + guides(scale="none") + ggtitle("d1_FGF1") + title_theme + black_background_theme
    d2_no_legend_with_title <- d2 + guides(scale="none") + ggtitle("d2_FGF1") + title_theme + black_background_theme

    # Combine the plots using patchwork with a black background
    combined_plots <- a1_with_title + c1_no_legend_with_title +
                      a2_no_legend_with_title + c2_no_legend_with_title +
                      b1_no_legend_with_title + d1_no_legend_with_title +
                      b2_no_legend_with_title + d2_no_legend_with_title +
                      plot_layout(guides = "collect", ncol = 2, heights = c(1, 1, 1, 1), widths = c(1, 1))
    
    # combined_plots = combined_plots & black_background_theme & legend_theme 
    combined_plots = combined_plots + plot_annotation(title=cell_type, theme=black_background_theme) + plot_annotation(theme=legend_theme) + plot_annotation(theme=title_theme)
    # Save the combined_plots to a PDF with a black background
    pdf_name = paste0(pdf_folder, cell_type, '_ct_mapping.pdf')
    pdf(pdf_name, width = 8.27, height = 11.69, bg = "black")
    # grid.newpage()
    grid.draw(combined_plots)
    dev.off()
}

In [18]:
cell_types = xenium.obj %>% `[[` %>% 
group_by(predicted.polar_label) %>% 
summarise(n = n()) %>% 
arrange(desc(n)) %>% 
pull(predicted.polar_label) %>% 
str_replace(fixed('.none'), '') %>%
str_replace(fixed('.pos'), '') %>%
str_replace(fixed('.neg'), '') %>%
unique
cell_types

[1] "Met__Prlh"           "Oligodendrocytes"    "Astrocytes"         
 [4] "Lepr15"              "OPC"                 "Esr1"               
 [7] "b2_Tanycytes"        "Lepr89"              "Agrp"               
[10] "Fez1"                "Dlk1"                "Nfib_1"             
[13] "a1_Tanycytes"        "Endothelial_cells"   "Gpr50__Tac2"        
[16] "Pomc__Anxa2"         "Pou6f2__Prkcq__Whrn" "Ppp1r17__Ltbp1"     
[19] "Pomc__Ttr"           "b1_Tanycytes"        "Lepr67"             
[22] "Trh__Cxcl12"         "Sst__Pthlh"          "Sst__Unc13c"        
[25] "Th__Slc6a3"          "Microglia"           "Ppp1r17__Tafa4"     
[28] "Nfib_3"              "Tmem215__Greb1"      "Sst__Nts"           
[31] "Htr3b"               "Lpar1_oligo"         "Ghrh"               
[34] "Nfix__Nfib"          "Reln__Lef1"          "Ependymal_cells"    
[37] "Kiss1__Tac2"         "Npsr1__Epha3"        "Th__Nfib"           
[40] "a2_Tanycytes"        "Met__Pappa2"         "Arx__Nr5a2"         
[43] "Ppp1r17"             "Nfib_2"              "Tbx19__Pirt"        
[46] "Pomc__Glipr1"        "Hdc"                 "Nfix__Nfib__Hgf"    
[49] "Nfix__Nfib__Lhx6"    "Slc17a6__Trhr"       "Trh__Lef1"          
[52] "Sim1"                "Irx5__Lmx1a"         "Tmem215__Dach2"

In [19]:
make_idp_pdf(xenium.obj, "OPC", prefix="", suffix="", pdf_folder="ct_mapping_rctd_polar_label/")

Warning message:
“Removing 25662 cells missing data for vars requested”
Warning message:
“Removing 25662 cells missing data for vars requested”
Warning message:
“Removing 25662 cells missing data for vars requested”
Warning message:
“Removing 25662 cells missing data for vars requested”
Warning message:
“Removing 25662 cells missing data for vars requested”
Warning message:
“Removing 25662 cells missing data for vars requested”
Warning message:
“Removing 25662 cells missing data for vars requested”
Warning message:
“Removing 25662 cells missing data for vars requested”


png 
  2

In [ ]:
for (cell_type in cell_types){
    print(cell_type)
    make_idp_pdf(xenium.obj, cell_type, pdf_folder="ct_mapping_rctd_polar_label/")
}

[1] "Met__Prlh"


Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message:
“Removing 25134 cells missing data for vars requested”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message:
“Removing 25134 cells missing data for vars requested”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message:
“Removing 25134 cells missing data for vars requested”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message:
“Removing 25134 cells missing data for vars requested”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message:
“Removing 25134 cells missing data for vars requested”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requeste

[1] "Oligodendrocytes"


Warning message:
“Removing 25211 cells missing data for vars requested”
Warning message:
“Removing 25211 cells missing data for vars requested”
Warning message:
“Removing 25211 cells missing data for vars requested”
Warning message:
“Removing 25211 cells missing data for vars requested”
Warning message:
“Removing 25211 cells missing data for vars requested”
Warning message:
“Removing 25211 cells missing data for vars requested”
Warning message:
“Removing 25211 cells missing data for vars requested”
Warning message:
“Removing 25211 cells missing data for vars requested”


[1] "Astrocytes"


...

Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message:
“Removing 27440 cells missing data for vars requested”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message:
“Removing 27440 cells missing data for vars requested”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message:
“Removing 27440 cells missing data for vars requested”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message:
“Removing 27440 cells missing data for vars requested”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message:
“Removing 27440 cells missing data for vars requested”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requeste

[1] "Endothelial_cells"


Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message:
“Removing 27697 cells missing data for vars requested”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message:
“Removing 27697 cells missing data for vars requested”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message:
“Removing 27697 cells missing data for vars requested”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message:
“Removing 27697 cells missing data for vars requested”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message:
“Removing 27697 cells missing data for vars requested”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requeste

[1] "Gpr50__Tac2"


Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message:
“Removing 27722 cells missing data for vars requested”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message:
“Removing 27722 cells missing data for vars requested”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message:
“Removing 27722 cells missing data for vars requested”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message:
“Removing 27722 cells missing data for vars requested”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message:
“Removing 27722 cells missing data for vars requested”
